# Hyperparameter Optimization

For this exercise, we will have a look at Hyperparameter Optimization --
instead of just choosing the best type of machine learning model, we also want
to choose the best hyperparameter setting for a task. The end result (i.e. the
predictive performance) is again not important; how you get there is.

Your deliverable will be a report, written in a style that it
would be suitable for inclusion in an academic paper as the "Experimental
Setup" section or similar. If unsure, check an academic paper of your choice,
for example [this one](https://www.eecs.uwyo.edu/~larsko/papers/pulatov_opening_2022-1.pdf). The
level of detail should be higher than in a typical academic paper though. Your
report should be at most five pages, including references and figures but
excluding appendices. It should have the following structure:
- Introduction: What problem are you solving, how are you going to solve it.
- Dataset Description: Describe the data you're using, e.g. how many features and observations, what are you predicting, any missing values, etc.
- Experimental Setup: What specifically are you doing to solve the problem, i.e.\ what programming languages and libraries, how are you processing the data, what machine learning algorithms are you considering and what hyperparameters and value ranges, what measures you are using to evaluate them, what hyperparameter optimization method you chose, etc.
- Results: Description of what you observed, including plots. Compare
  performance before and after tuning, and show the best configuration.
- Code: Add the code you've used as a separate file.

Your report must contain enough detail to reproduce what you did without the
code. If in doubt, include more detail.

There is no required format for the report. You could, for example, use an
iPython notebook.

## Data and Setup

We will have a look at the [Wine Quality
dataset](https://archive-beta.ics.uci.edu/dataset/186/wine+quality). Choose the
one that corresponds to your preference in wine. You may also use a dataset of
your choice, for example one that's relevant to your research.

Choose a small number of different machine learning algorithms and
hyperparameters, along with value ranges, for each. You can use implementations
of AutoML systems (e.g. auto-sklearn), scientific papers, or the documentation
of the library you are using to determine the hyperparameters to tune and the
value ranges. Note that there is not only a single way to do this, but define a
reasonable space (e.g. don't include whether to turn on debug output, or random
forests with 1,000,000 trees, or tune the loss function). Your hyperparameter
search space should be so large that you cannot simply run a grid search.

Determine the best machine learning algorithm and hyperparameter setting for
your dataset. Make sure to optimize both the type of machine learning algorithm
and the hyperparameters at the same time (do not first choose the best ML
algorithm and then optimize its hyperparameters). Choose a suitable
hyperparameter optimizer; you could also use several and e.g. compare the
results achieved by random search and Bayesian optimization. Make sure that the
way you evaluate model performance avoids bias and overfitting. You could use
statistical tests to make this determination.

## Submission

Add your report and code to this repository. Bonus points if you can set up a
Github action to automatically run the code and generate the report!

## Useful Resources :
- "*Basics of HPO - Example and Practical Hints*" -From the AutoML Course Videos
- https://www.youtube.com/watch?v=Gol_qOgRqfA
- https://www.youtube.com/watch?v=0wUF_Ov8b0A&t=1058s

## Importing the Dataset as a Pandas Dataframe

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
red_wine_df = pd.read_csv('winequality-red.csv', delimiter=';')

In [ ]:
red_wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
X = red_wine_df.iloc[:, :-1]
y = red_wine_df['quality']

X.shape, y.shape

((1599, 11), (1599,))

## Importing our Model (KNN Classifier)

### K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()

In [ ]:
knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

## Needed Data Pre-processing for KNN

In [ ]:
# RESOURCE : https://arize.com/blog-course/knn-algorithm-k-nearest-neighbor/#:~:text=Preprocessing%20Data&text=Scaling%20the%20data%20is%20an,can%20result%20in%20incorrect%20classifications.
# Preprocess the data for the KNN model:

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

## Hyperparameter Optimization

Methods Used :
- Bayesian Optimization
- Random Search

### Bayesian Optimization

In [ ]:
# Comment out this line to install the necessary library for Bayesian Optimization:
!pip install baytune

In [ ]:
models = {
    'KNN': KNeighborsClassifier,
}

In [ ]:
from sklearn.model_selection import cross_val_score

def scoring_function(model_name, hyperparameter_values):
    model_class = models[model_name]
    model_instance = model_class(**hyperparameter_values)
    scores = cross_val_score(
        cv=10,
        estimator=model_instance,
        X=X_scaled,
        y=y,
        scoring='accuracy',
    )

    return scores.mean()

In [ ]:
from baytune.tuning import Tunable
from baytune.tuning import hyperparams as hp

tunables = {
    'KNN': Tunable({
        'n_neighbors': hp.IntHyperParam(min=1, max=1000, default=5),
        'weights' : hp.CategoricalHyperParam(choices=['uniform', 'distance'], default='uniform'),
        'algorithm' : hp.CategoricalHyperParam(choices=['auto', 'ball_tree', 'kd_tree', 'brute'], default='auto'),
        'leaf_size' : hp.IntHyperParam(min=1, max=100000, default=30),
        'p' : hp.IntHyperParam(min=1, max=100000, default=2)
    }),
}

In [ ]:
from baytune import BTBSession

session = BTBSession(
    tunables=tunables,
    scorer=scoring_function,
    verbose=True,
)

In [ ]:
best_result = session.run(50)

  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The opti

In [ ]:
best_result

{'id': '804ea6e0183e7138ddb10c8165e4eabc',
 'name': 'KNN',
 'config': {'n_neighbors': 241,
  'weights': 'distance',
  'algorithm': 'kd_tree',
  'leaf_size': 39403,
  'p': 836},
 'score': 0.5584669811320755}

### Random Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

#### K-Nearest Neighbors

In [ ]:
# Define the hyperparameters:

k_neighbors = range(1, 1000)
weights = ['uniform', 'distance']
algorithms = ['auto', 'ball_tree', 'kd_tree', 'brute']
leaf_sizes = range(1, 100000)
p_values = range(1, 100000)

In [ ]:
# Construct the hyperparameter distribution:

hyperparameter_distribution = {
    "n_neighbors" : k_neighbors,
    "weights" : weights,
    "algorithm" : algorithms,
    "leaf_size" : leaf_sizes,
    "p" : p_values
}

In [ ]:
# Construct the "Random Search" object:

K_FOLDS = 10
ITERATIONS = 200
random_search = RandomizedSearchCV(knn_model, hyperparameter_distribution, cv=K_FOLDS, scoring='accuracy', n_iter=ITERATIONS, verbose=4)

In [ ]:
# Run the "Random Search" on the dataset and on the hyperparameter distribution:

random_search.fit(X_scaled, y)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits
[CV 1/10] END algorithm=auto, leaf_size=70081, n_neighbors=233, p=65145, weights=uniform;, score=0.044 total time=   0.1s
[CV 2/10] END algorithm=auto, leaf_size=70081, n_neighbors=233, p=65145, weights=uniform;, score=0.425 total time=   0.3s
[CV 3/10] END algorithm=auto, leaf_size=70081, n_neighbors=233, p=65145, weights=uniform;, score=0.425 total time=   0.2s
[CV 4/10] END algorithm=auto, leaf_size=70081, n_neighbors=233, p=65145, weights=uniform;, score=0.425 total time=   0.1s
[CV 5/10] END algorithm=auto, leaf_size=70081, n_neighbors=233, p=65145, weights=uniform;, score=0.425 total time=   0.1s
[CV 6/10] END algorithm=auto, leaf_size=70081, n_neighbors=233, p=65145, weights=uniform;, score=0.425 total time=   0.1s
[CV 7/10] END algorithm=auto, leaf_size=70081, n_neighbors=233, p=65145, weights=uniform;, score=0.425 total time=   0.1s
[CV 8/10] END algorithm=auto, leaf_size=70081, n_neighbors=233, p=65145, weights=

RandomizedSearchCV(cv=10, estimator=KNeighborsClassifier(), n_iter=200,
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'leaf_size': range(1, 100000),
                                        'n_neighbors': range(1, 1000),
                                        'p': range(1, 100000),
                                        'weights': ['uniform', 'distance']},
                   scoring='accuracy', verbose=4)

In [ ]:
# Check the results:

pd.DataFrame(random_search.cv_results_)[['mean_test_score', 'std_test_score', 'params']]

,mean_test_score,std_test_score,params
0,0.387142,0.114467,"{'weights': 'uniform', 'p': 65145, 'n_neighbor..."
1,0.386517,0.116342,"{'weights': 'uniform', 'p': 93676, 'n_neighbor..."
2,0.386517,0.116342,"{'weights': 'uniform', 'p': 12714, 'n_neighbor..."
3,0.389017,0.111010,"{'weights': 'uniform', 'p': 53559, 'n_neighbor..."
4,0.386517,0.116342,"{'weights': 'uniform', 'p': 65985, 'n_neighbor..."
...,...,...,...
195,0.386517,0.116342,"{'weights': 'uniform', 'p': 73270, 'n_neighbor..."
196,0.388392,0.110717,"{'weights': 'uniform', 'p': 19411, 'n_neighbor..."
197,0.486568,0.075547,"{'weights': 'distance', 'p': 62793, 'n_neighbo..."
198,0.540334,0.043082,"{'weights': 'uniform', 'p': 114, 'n_neighbors'..."


In [ ]:
print(random_search.best_score_)
print(random_search.best_params_)

0.5459551886792453
{'weights': 'uniform', 'p': 168, 'n_neighbors': 403, 'leaf_size': 18717, 'algorithm': 'ball_tree'}
